In [3]:
# PJ LL데이터에 사례 내용 데이터 "사례번호" 기준으로 통합

import pandas as pd

# 엑셀 파일 두 개 가져오기
df1 = pd.read_excel("./data/Lessons Learned_title.xlsx", engine='openpyxl')
df2 = pd.read_excel("./data/preprocessed_data.xlsx", engine='openpyxl')

df2_filtered = df2[['사례번호', 'content']]

# 데이터 조인
df = pd.merge(df1, df2_filtered, on="사례번호", how='left')

df.to_excel("./data/content_merged.xlsx", index=False, engine='openpyxl')

df.tail()

,No,구분,중요도,본부,업무프로세스,사업분야,공종,사례번호,사례제목,승인일,현장/입찰PJ명,사례담당자 사번,사례담당자,등록인 사번,등록인,대륙,국가,content
106,110,실패,Low,플랜트,수행,발전,수행설계-계장,20-00014,FGC LSB 방폭 적용 건,2020.01.15,신평택복합화력주기기PJ(현장),NaN,NaN,1200121,김경은,국내,대한민국,"구분: 실패, 사례제목: FGC LSB 방폭 적용 건, 현장/입찰PJ명: 신평택복합..."
107,111,실패,Low,플랜트,수행,발전,수행구매-기계,20-00012,구매계약 이후 공급 범위 변경 건,2020.01.13,신평택복합화력주기기PJ(현장),NaN,NaN,1101067,김용남,국내,대한민국,"구분: 실패, 사례제목: 구매계약 이후 공급 범위 변경 건, 현장/입찰PJ명: ..."
108,112,우수,Low,플랜트,수행,발전,수행설계-계장,20-00011,2복합(타 발전소) 연계신호 반영의 건,2020.01.13,신평택복합화력주기기PJ(현장),1102454.0,이기훈,1102454,이기훈,국내,대한민국,"구분: 우수, 사례제목: 2복합(타 발전소) 연계신호 반영의 건, 현장/입찰PJ명:..."
109,113,실패,Low,플랜트,수행,발전,수행구매-기계,20-00006,Fuel Gas Treatment_Supervisor 요청 및 취소 건,2020.01.10,신평택복합화력주기기PJ(현장),NaN,NaN,1101067,김용남,국내,대한민국,"구분: 실패, 사례제목: Fuel Gas Treatment_Supervisor 요청..."
110,114,실패,Low,플랜트,수행,발전,수행설계-계장,20-00002,실외 impulse tubing 계측기 동파발생,2020.01.06,신평택복합화력주기기PJ(현장),NaN,NaN,1100292,박현철,국내,대한민국,"구분: 실패, 사례제목: 실외 impulse tubing 계측기 동파발생, 현장/입..."


In [11]:
import configparser
from langchain_openai import ChatOpenAI

config = configparser.ConfigParser()
config.read('config.ini')
openai_api_key = config['openai']['api_key']

In [22]:
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import time 

# OpenAI GPT 모델 초기화
model = ChatOpenAI(model = "gpt-4-turbo-preview", openai_api_key=openai_api_key)

def define_chains():
    # 체인 정의
    system_message = "한국의 발전소의 주기기를 구매하는 Project의 입찰 리스크 분석을 위해 과거의 경험데이터를 분석한다.\n건설의 용어, 약어를 잘 활용하고, 건설 각 분야 전문가들에게 배포할 정보이기 때문에 전문적으로 작성되어져야 한다."
    #prompt1 = ChatPromptTemplate.from_template("{case_content}   >> 위의 사례 데이터를 제공할테니,  200자로 요약해줘. 핵심적인 내용이 들어가야하고 빠르게 신규 프로젝트 입찰에 대해 리스크를 예측하고 분석, 판단하기 좋게 요약해줘.")
    prompt1 = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("user", "{case_content} >> 위의 사례 데이터를 제공할테니,  250자로 요약해줘. 핵심적인 내용이 들어가야하고 빠르게 신규 프로젝트 입찰에 대해 리스크를 예측하고 분석, 판단하기 좋게 요약해줘.")
    ])

    prompt2 = ChatPromptTemplate.from_template("{case_content}   이 사례를 통해 얻은 교훈에 대해 최대 3가지로 dotted list로 표기해줘. 최대한 간략하게")
    prompt3 = ChatPromptTemplate.from_template("{case_content}   본 사례에서 얻은 회사의 이득, 손실이나, 수익, 비용에 대한 수치를 짧게 단어로 표기해줘")
    prompt4 = ChatPromptTemplate.from_template("{case_content}   본 사례의 주요 키워드를 작성하는데, 신규 입찰 참여 리스크 검토자들이 사례를 보지 않고도 키워드만보고 어떤 사례이구나 알 수 있게 최소 3개 최대 10개 작성해줘.")

    # 체인을 모델과 파서와 연결
    chain1 = prompt1 | model | StrOutputParser()
    chain2 = prompt2 | model | StrOutputParser()
    chain3 = prompt3 | model | StrOutputParser()
    chain4 = prompt4 | model | StrOutputParser()
    
    return chain1, chain2, chain3, chain4

def process_row(row):
    chain1, chain2, chain3, chain4 = define_chains()
    # 체인 실행
    response1 = chain1.invoke({"case_content": row['content']})
    time.sleep(0.5)
    response2 = chain2.invoke({"case_content": row['content']})
    time.sleep(0.5)
    response3 = chain3.invoke({"case_content": row['content']})
    time.sleep(0.5)
    response4 = chain4.invoke({"case_content": row['content']})
    time.sleep(0.5)

    return response1, response2, response3, response4
    
df_sample = df[-2:]

#responses = df.apply(process_row, axis=1)
responses = df_sample.apply(process_row, axis=1)

df_sample['Ai 사례요약'] = responses.apply(lambda x: x[0])
df_sample['Lessons Learned'] = responses.apply(lambda x: x[1])
df_sample['득실'] = responses.apply(lambda x: x[2])
df_sample['키워드'] = responses.apply(lambda x: x[3])


# df['Ai 사례요약'] = responses.apply(lambda x: x[0])
# df['Lessons Learned'] = responses.apply(lambda x: x[1])
# df['득실'] = responses.apply(lambda x: x[2])
# df['키워드'] = responses.apply(lambda x: x[3])

# case_content = """ 
# 구분: 실패, 사례제목: 미 검증된 Vendor와 계약체결_3, 현장/입찰PJ명: 인천논현집단에너지시설현장, 사례내용: - 보일러 급수펌프 납기지연 (약7개월)       - 계약납기: 20061020       - 조정납기: 20061230       - 실입고일: 2007621- 유체카플링 부품추가 장착- 납기지연 요인       - 유체카플링에 온도계, 오일레벨스위치 추가(독일 VOITH)- 1개월       - 전동기 냉각팬의 회전날개 오제작- 1개월       - 밸런스베어링과 슬리브의 결함(발열에 의한 외피손상)- 4개월       - 펌프 BED 오작- 1개월       - 베어링 하우징과 슬리브베어링의 동심도 허용치 초과- 1개월        - 국내 최초로 발전 설비용 급수펌프의 국산화에는 성공했으나 종합준공 시운전준비에 영향을 초래하여 - 최초에 Equipment 발주시 정확한 스펙으로 발주 할 것
# """



In [ ]:
# 결과를 엑셀 파일로 저장
df.to_excel("output.xlsx", index=False, engine='openpyxl')